# Week 2 Overview

During this week's lessons, you will learn how the vector space model works in detail, the major heuristics used in designing a retrieval function for ranking documents with respect to a query, and how to implement an information retrieval system (i.e., a search engine), including how to build an inverted index and how to score documents quickly for a query.

## Key Phrases and Concepts

Keep your eyes open for the following key terms or phrases as you complete the readings and interact with the lectures. These topics will help you better understand the content in this module.

* Term frequency (TF)
* Document frequency (DF) and inverse document frequency (IDF)
* TF transformation
* Pivoted length normalization
* BM25
* Inverted index and postings
* Binary coding, unary coding, gamma-coding, and d-gap
* Zipf’s law 

## Goals and Objectives

After you actively engage in the learning experiences in this module, you should be able to:

* Explain what TF-IDF weighting is and why TF transformation and document length normalization are necessary for the design of an effective ranking function.
* Explain what an inverted index is and how to construct it for a large set of text documents that do not fit into the memory.
* Explain how variable-length encoding can be used to compress integers and how unary coding and gamma-coding work.
* Explain how scoring of documents in response to a query can be done quickly by using an inverted index.
* Explain Zipf’s law.

## Guiding Questions

Develop your answers to the following guiding questions while completing the readings and working on assignments throughout the week.

* What are some different ways to place a document as a vector in the vector space?
    1. Represent words as bit-vectors \\(\{w_i, ..., w_n\} \ w_i \in {0, 1}\\) where 1 for mathced word to query and 0 for unmatched word to query.
    2. Represent words as matched words count vectors \\(c(W_i, d)\\) .
    3. Represent words as penalized matched words count vectors using IDF function \\(c(W_i, d)*IDF(W_i)\\).
<br><br>
* What is term frequency (TF)?<br>
Count of word occurencess in the specific document.
<br><br>
* What is TF transformation?<br>
Transform TF value into desired scale, such as:
    1. Linear scale difined by word count.
    2. Sublinear scale, i.e Logarithmic scale and BM25.
    3. Bit scale.
<br><br>
* What is document frequency (DF)?<br>
Document frequency is sum of documents contain term \\(t_i\\). 
<br><br>
* What is inverse document frequency (IDF)?<br>
Inverse document frequency is logarithm of number of document devided by document frequency.
<br><br>
* What is TF-IDF weighting?<br>
TF-IDF weighting tells us that in large document collections, common words should have low rank value because they are tend to be biased by statistical point of view and rare words should have high rank value because they intuitively can be used to differentiate document by topic.
<br><br>
* Why do we need to penalize long documents in text retrieval?<br>
In the case if bit-vector representation \\(\{w_i, ..., w_n\} \ w_i \in {0, 1}\\) not staisfy information need, we may used word count representation \\(c(W_i, d)\\). Then statistically speaking, we expected that TF should be uniformly distributed over related documents. If not uniformly distributed, then very high TF in few documents could skewed word distribution. In the skewed distribution, even conventional tf-idf function \\(c(W_i, d)*IDF(W_i)\\) may not helpful in order to generate fair relevance rank.
<br><br>
* What is pivoted document length normalization?<br>
Pivoted length normalizer help us to normalize document length. The basic normalization work in two steps: First, it will normalize document length by divide document frequency \\(DF\\) with average document frequency \\(\overline{DF}\\) and second, penalized normalized document length over desired upper bound \\(b\\).
![pivoted-length-normalizer](images/pivoted-length-normalizer.png)<br>
Then, we can impliment the document length normalization in TF-IDF calculation. In the case if we used logaithmic TF-IDF, then the our document vector will calculated by this formula:<br>
$$f(q, d) = \sum_{w \in q \cap d} c(w,q)*\frac{ln[1+ln[1+c(w,d)]]}{1-b+b\frac{|d|}{\overline{dl}}}*log\frac{M+1}{df(w)}$$
<br><br>
* What are the main ideas behind the retrieval function BM25?<br>
Basically, BM25 help us to penalize very high TF value by using upper bound value \\(k\\). It can also be used with document length normalization:<br>
$$f(q, d)=\sum_{w \in q \cap d} c(w,q)*\frac{(k+1)c(w,d)}{c(w,d)+k(1-b+b \frac{|d|}{\overline{dl}})}*log \frac{M+1}{df(w)}$$
where:
$$\eqalign{
b &\in [0,1]\\
k_1, k_3 &\in [0, +\infty)
}$$
<br><br>
* What is the typical architecture of a text retrieval system?<br>
A typical architecture of a text retrieval consisted by four components: **Tokenizer**, **Indexer**, **Scorer**, and **Feedback**.

![typical architecture](images/typical-architecture.png)
<br><br>
* What is an inverted index?<br>
Inverted index is indexing strategy to provide fast document lookup and aggregation process such as TF-IDF. Basically, it consisted by two elements: **Dictionary** and **Postings**. An image below is an example of inverted index which suited for TF-IDF model:
![inverted index](images/inverted-index.png)
<br><br>
* Why is it desirable to compress an inverted index?<br>
If we working with large corpus, then **memory-based method** is not feasible. Another way is using **sort-based method** with following steps:
    1. Collect local \\((termID, docID, freq)\\) tuples.
    2. Sort local tuples (to make runs)
    3. Pair-wise merge runs
    4. Output inverted index
All step above could be illustrate in an image below:
![sort based method](images/sort-based-method.png)<br>
The critical part of this method is **ensure that local sort and merge sort** fit in memory. To do that, we can create virtual memory with enough space. **In the case that virtual memory does not satisfy performance requirement, then we should implement compression algorithm to compress tuple(termID, docID, freq)**.
Another reason we need compression algorithm is **distribution of term frequency (TF) tend to follow zipf's law** that tells us that most of words have low frequency, but only few words have very high frequency. This fact will caused distribution of trem frequencies tend to be skewed. Furthermore, **dirtibution of a term in documents with general topic** may also skewed, few words may occured in most documents, while most words occured in few documents.
<br><br>
* How can we create an inverted index when the collection of documents does not fit into the memory?<br>
We can implement coding theory and information theory to encode distribution of term frequncy and document id:
    1. **TF compression**: Fewer bits for high frequency term and more bits for low frequncy term.
    2. **Doc ID compression**: Use d-gap compression that store first mathced document follwed by distance to next Doc ID. For example, term found in Doc ID [d1, d4, d9, d10] can be compressed as {d1: [3, 5, 1]}.
<br><br>
* How can we leverage an inverted index to score documents quickly?<br>
The idea of how we can leverage an inverted index to score documents quickly is by using **data aggregation** into inverted index. Formally, we can computer score function of each document by using this formula:
$$f(q,d) = f_a(h(g(t_1,d,q), ..., g(t_k,d,q)), f_d(d), f_q(q))$$
where:
$$\eqalign{
f_a &: \text{Final score adjustment}\\
f_d(d) &: \text{Document score adjustment}\\
f_q(q) &: \text{Query score adjustment}\\
h &: \text{Weight aggregation}\\
g(t_i,d,q) &: \text{Weight of mathed query term in d}
}$$<br>
A general algorithm to implement score function in order to ranking documents decribed in folloing steps:
    1. Pre-compute \\(f_d(d)\\) and \\(f_q(q)\\).
    2. Maintain a score accumulator for each \\(d\\) to compute \\(h\\).
    3. For each query term \\(t_i\\):
        - Fetch the inverted list \\(\{(d_1,f_1),..,(d_n,f_n)\}\\)
        - For each entry \\((d_j,f_j)\\), computer \\(g(t_i,d_j,1)\\), and update score accumulator for doc \\(d_i\\) to incrementally compute \\(h\\)
    4. Adjust the score to compute \\(f_a\\) and sort
<br><br>

## Additional Readings and Resources

The following readings are optional:

* C. Zhai and S. Massung. Text Data Management and Analysis: A Practical Introduction to Information Retrieval and Text Mining, ACM Book Series, Morgan & Claypool Publishers, 2016. Chapter 6 - Section 6.3, and Chapter 8.
* Ian H. Witten, Alistair Moffat, and Timothy C. Bell. Managing Gigabytes: Compressing and Indexing Documents and Images, Second Edition. Morgan Kaufmann, 1999.
* Stefan Büttcher, Charles L. A. Clarke, Gordon V. Cormack: Information Retrieval - Implementing and Evaluating Search Engines. MIT Press, 2010.